In [ ]:
# Grammarly Analytics the Messy Way


# Importing required libraries
from apiclient import discovery
from apiclient import errors
from httplib2 import Http
from oauth2client import file, client, tools
import base64
from bs4 import BeautifulSoup
from lxml.html.soupparser import fromstring
from lxml import etree
from lxml import html
from lxml.html.clean import clean_html
import re
import time
import dateutil.parser as parser
from datetime import datetime
import datetime
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
"""
Shows basic usage of the Gmail API.

Lists the user's Gmail labels.
"""
from __future__ import print_function
from apiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

# Setup the Gmail API
SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'
store = file.Storage('credentials.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = build('gmail', 'v1', http=creds.authorize(Http()))

# Call the Gmail API
results = service.users().labels().list(userId='me').execute()
labels = results.get('labels', [])
if not labels:
    print('No labels found.')
else:
    print('Labels:')
    for label in labels:
        print(label['name'])

In [ ]:


'''
Reading GMAIL using Python
	- Abhishek Chhibber
    - https://github.com/abhishekchhibber/Gmail-Api-through-Python
    - Modified by Joshua Ansell-McKinnon
'''

'''
This script does the following:
- Go to Gmal inbox
- Find and read all the unread messages
- Extract details (Date, Sender, Subject, Snippet, Body) and export them to a .csv file / DB
- Mark the messages as Read - so that they are not read again 
'''

'''
Before running this script, the user should get the authentication by following 
the link: https://developers.google.com/gmail/api/quickstart/python
Also, client_secret.json should be saved in the same directory as this file
'''


# Creating a storage.JSON file with authentication details
SCOPES = 'https://www.googleapis.com/auth/gmail.modify' # we are using modify and not readonly, as we will be marking the messages Read
store = file.Storage('storage.json') 
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store)
GMAIL = discovery.build('gmail', 'v1', http=creds.authorize(Http()))

user_id =  'me'
label_id_one = 'INBOX'
label_id_two = 'UNREAD'
# query I used
query = "in:anywhere subject:(Your Weekly Progress Report)"

# Getting all the unread messages from Inbox
# labelIds can be changed accordingly
# unread_msgs = GMAIL.users().messages().list(userId='me',labelIds=[label_id_one, label_id_two]).execute()
unread_msgs = GMAIL.users().messages().list(userId='me',q=query).execute()


# We get a dictonary. Now reading values for the key 'messages'
mssg_list = unread_msgs['messages']

print ("Total unread messages in inbox: ", str(len(mssg_list)))

final_list = [ ]

name_file_number = 1

for mssg in mssg_list:
	temp_dict = { }
	m_id = mssg['id'] # get id of individual message
	message = GMAIL.users().messages().get(userId=user_id, id=m_id).execute() # fetch the message using API
	payld = message['payload'] # get payload of the message 
	headr = payld['headers'] # get header of the payload


	for one in headr: # getting the Subject
		if one['name'] == 'Subject':
			msg_subject = one['value']
			temp_dict['Subject'] = msg_subject
		else:
			pass


	for two in headr: # getting the date
		if two['name'] == 'Date':
			msg_date = two['value']
			date_parse = (parser.parse(msg_date))
			m_date = (date_parse.date())
			temp_dict['Date'] = str(m_date)
		else:
			pass

	for three in headr: # getting the Sender
		if three['name'] == 'From':
			msg_from = three['value']
			temp_dict['Sender'] = msg_from
		else:
			pass

	temp_dict['Snippet'] = message['snippet'] # fetching message snippet


	try:
		
		# Fetching message body
		mssg_parts = payld['parts'] # fetching the message parts
		part_one_before  = mssg_parts[1] # fetching first element of the part
		part_one  = mssg_parts[0] # fetching first element of the part 
		part_body = part_one['body'] # fetching body of the message
		part_data = part_body['data'] # fetching data from the body
		clean_one = part_data.replace("-","+") # decoding from Base64 to UTF-8
		clean_one = clean_one.replace("_","/") # decoding from Base64 to UTF-8
		clean_two = base64.b64decode (bytes(clean_one, 'UTF-8')) # decoding from Base64 to UTF-8
		soup = BeautifulSoup(clean_two , "lxml" )
		mssg_body = soup.body()

		# mssg_body is a readible form of message body
		# depending on the end user's requirements, it can be further cleaned 
		# using regex, beautiful soup, or any other method
		temp_dict['Message_body'] = mssg_body
		#print(temp_dict['Message_body'])
		#s = mssg_body
		        
		
	except :
		pass
    

# 	# This will save your emails to a file
# 	name_file = "email_" + str(name_file_number)
# 	file = open("messages/" + "%s" % name_file + ".html" , 'w') 
# 	file.write(str(mssg_body))
# 	name_file_number += 1
# 	print(name_file_number)
	
	final_list.append(temp_dict) # This will create a dictonary item in the final list
	
	# This will mark the messagea as read
	# GMAIL.users().messages().modify(userId=user_id, id=m_id,body={ 'removeLabelIds': ['UNREAD']}).execute() 
	



print ("Total messaged retrived: ", str(len(final_list)))

'''

The final_list will have dictionary in the following format:

{	'Sender': '"email.com" <name@email.com>', 
	'Subject': 'Lorem ipsum dolor sit ametLorem ipsum dolor sit amet', 
	'Date': 'yyyy-mm-dd', 
	'Snippet': 'Lorem ipsum dolor sit amet'
	'Message_body': 'Lorem ipsum dolor sit amet'}


The dictionary can be exported as a .csv or into a databse
'''

#exporting the values as .csv
with open('CSV_NAME.csv', 'w', encoding='utf-8', newline = '') as csvfile: 
    fieldnames = ['Sender','Subject','Date','Snippet','Message_body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter = ',')
    writer.writeheader()
    for val in final_list:
    	writer.writerow(val)
    

In [ ]:
#See all of your dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv("CSV_NAME.csv")

In [ ]:
# This can use some clean up and will change as Grammarly updates its emails
# Anywhere you see .loc[] you will have to match youp email, which reminds me this was made for my emails from Grammarly

# Clean words checked
c = 'words checked'
df['words_checked_start'] = df['Message_body'].str.find(c)
fn_slice_hit = lambda x : x['Message_body'][x['words_checked_start']-20:x['words_checked_start']]
#apply the slice function to the dataframe
df['sliced'] = df.apply(fn_slice_hit, axis = 1)

fn_slice_hit_three = lambda x : x['sliced'][:10]
df['sliced'].loc[35:] = df.apply(fn_slice_hit_three, axis = 1)

df['sliced'] = df['sliced'].str.replace(r"[\r\n,;& ]",'')
df['sliced'] = df['sliced'].str.replace(r"[a-zA-Z]",'')

df.sliced = pd.to_numeric(df.sliced, errors='coerce').fillna(0).astype(np.int64)

df.rename(columns={"sliced": "Words Checked"}, inplace=True)

In [ ]:
# Clean alerts shown
df['alerts_shown_start'] = df['Message_body'].str.find('alerts shown')
fn_slice_hit = lambda x : x['Message_body'][x['alerts_shown_start']-20:x['alerts_shown_start']]
#apply the slice function to the dataframe
df['alerts_shown_sliced'] = df.apply(fn_slice_hit, axis = 1)
df['alerts_shown_sliced'].loc[:35] = df.apply(fn_slice_hit, axis = 1)

# Clean corrections made
df['alerts_shown_start_two'] = df['Message_body'].str.find('corrections made')
fn_slice_hit_two = lambda x : x['Message_body'][x['alerts_shown_start_two']-20:x['alerts_shown_start_two']-10]
#apply the slice function to the dataframe
df['alerts_shown_sliced'].loc[35:] = df.apply(fn_slice_hit_two, axis = 1)
df['alerts_shown_sliced']

In [ ]:
df['alerts_shown_sliced'] = df['alerts_shown_sliced'].str.replace(r"[\r\n,;& ]",'')
df['alerts_shown_sliced'] = df['alerts_shown_sliced'].str.replace(r"[a-zA-Z]",'')
df['alerts_shown_sliced']
df.alerts_shown_sliced = pd.to_numeric(df.alerts_shown_sliced, errors='coerce').fillna(0).astype(np.int64)

df.rename(columns={"alerts_shown_sliced": "Alerts Shown"}, inplace=True)

In [ ]:
# Clean unique words used
c = 'unique words used'
df['unique_words_start'] = df['Message_body'].str.find(c)
fn_slice_hit_two = lambda x : x['Message_body'][x['unique_words_start']-15:x['unique_words_start']]
#apply the slice function to the dataframe
df['unique_words_sliced'] = df.apply(fn_slice_hit_two, axis = 1)
df['unique_words_sliced']

df['unique_words_sliced'] = df['unique_words_sliced'].str.replace(r"[\r\n,;& ]",'')
df['unique_words_sliced'] = df['unique_words_sliced'].str.replace(r"[a-zA-Z]",'')
df.unique_words_sliced = pd.to_numeric(df.unique_words_sliced, errors='coerce').fillna(0).astype(np.int64)

df.rename(columns={"unique_words_sliced": "Unique Words"}, inplace=True)

In [ ]:
df

In [ ]:
df2 = df[['Date','Words Checked','Alerts Shown', 'Unique Words']]

In [ ]:
df2.plot(subplots=True, figsize=(6, 6)); plt.legend(loc='best')

In [ ]:
df2['Words vs Alerts'] = df2['Words Checked']//df2['Alerts Shown']

In [ ]:
df2

In [ ]:
#df2.set_index(['Date'],inplace=True)
df2['Date'] = pd.to_datetime(df.Date)
df2.sort_values(by='Date')
df2.set_index(['Date'],inplace=True)
#df2['Words vs Alerts'].plot(subplots=True, figsize=(6, 6)); plt.legend(loc='best')

In [ ]:
df2.plot(subplots=True, figsize=(6, 6)); plt.legend(loc='best')

In [ ]:
df2['Words vs Alerts'].plot(subplots=True, figsize=(6, 6)); plt.legend(loc='best')